In [1]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install jsonlines --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00


In [2]:
from transformers import (
    BertTokenizer,
    BertModel,
    TFBertModel,
    BertConfig,
    AutoTokenizer,
    AutoModel,
)

import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = '/content/drive/MyDrive/w266_final_project/data/RECCON/'

In [6]:
# Dataset doesn't provide an explicit "train" set, we can attempt to partition the existing data
df_train = pd.read_csv(f'{DATA_PATH}/dailydialog_qa_train_without_context.csv')
df_valid = pd.read_csv(f'{DATA_PATH}/dailydialog_qa_valid_without_context.csv')
df_test = pd.read_csv(f'{DATA_PATH}/dailydialog_qa_test_without_context.csv')

In [7]:
(len(df_train), len(df_valid), len(df_test))

(14627, 655, 3705)

In [10]:
df_train_pos = df_train[df_train.cause_span != 'Impossible ≠æ≠æ≠ answer']
df_train_pos

,id,emotion,utterance,cause_utterance,cause_span
0,dailydialog_tr_1097_utt_1_true_cause_utt_1_span_0,happiness,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...
3,dailydialog_tr_1097_utt_3_true_cause_utt_3_span_0,happiness,You have no idea ! There's a banquet with priz...,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances"
6,dailydialog_tr_1097_utt_4_true_cause_utt_3_span_0,surprise,Really ? What kind of prizes ?,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances"
9,dailydialog_tr_1097_utt_5_true_cause_utt_5_span_0,happiness,"Well , I heard that last year Vivian from acco...","Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !
12,dailydialog_tr_1097_utt_6_true_cause_utt_5_span_0,surprise,A new car ! You're kidding !,"Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !
...,...,...,...,...,...
14616,dailydialog_tr_943_utt_2_true_cause_utt_2_span_0,disgust,I hate school !,I hate school !,I hate school !
14619,dailydialog_tr_943_utt_3_true_cause_utt_2_span_0,surprise,why ? What happened ?,I hate school !,I hate school !
14622,dailydialog_tr_943_utt_4_true_cause_utt_4_span_0,disgust,I messed up my mid-term exam so badly today !,I messed up my mid-term exam so badly today !,I messed up my mid-term exam so badly today !
14625,dailydialog_tr_943_utt_6_true_cause_utt_6_span_0,disgust,"yes , but it's so humiliating ! I don't want m...","yes , but it's so humiliating ! I don't want m...",it's so humiliating !


In [11]:
df_valid_pos = df_valid[df_valid.cause_span != 'Impossible ≠æ≠æ≠ answer']
df_valid_pos

,id,emotion,utterance,cause_utterance,cause_span
2,dailydialog_va_922_utt_6_true_cause_utt_5_span_0,surprise,Are you crazy ? You have no idea what you are ...,Come on ! We have been here for almost an hour...,I thought you said you were only going to get ...
3,dailydialog_va_922_utt_6_true_cause_utt_6_span_1,surprise,Are you crazy ? You have no idea what you are ...,Are you crazy ? You have no idea what you are ...,You have no idea what you are talking about ! ...
6,dailydialog_va_922_utt_7_true_cause_utt_6_span_0,anger,Whoa whoa whoa ! Are you nuts ? How much is al...,Are you crazy ? You have no idea what you are ...,"Just for my eyes I have to get eye-liner , an ..."
7,dailydialog_va_922_utt_7_true_cause_utt_7_span_1,anger,Whoa whoa whoa ! Are you nuts ? How much is al...,Whoa whoa whoa ! Are you nuts ? How much is al...,How much is all this going to cost ? I ' m loo...
10,dailydialog_va_922_utt_8_true_cause_utt_7_span_0,anger,What about you ? You spend as much or more mon...,Whoa whoa whoa ! Are you nuts ? How much is al...,Are you nuts ? How much is all this going to c...
...,...,...,...,...,...
645,dailydialog_va_472_utt_1_true_cause_utt_1_span_0,happiness,I was really glad to hear about your award . C...,I was really glad to hear about your award . C...,to hear about your award .
647,dailydialog_va_472_utt_2_true_cause_utt_1_span_0,happiness,"Thanks , Denise . Actually , I was really surp...",I was really glad to hear about your award . C...,I was really glad to hear about your award . C...
648,dailydialog_va_472_utt_2_true_cause_utt_1_span_1,happiness,"Thanks , Denise . Actually , I was really surp...",I was really glad to hear about your award . C...,award
651,dailydialog_va_472_utt_3_true_cause_utt_1_span_0,happiness,Sure . But the work you did was really excepti...,I was really glad to hear about your award . C...,to hear about your award .


In [12]:
df_test_pos = df_test[df_test.cause_span != 'Impossible ≠æ≠æ≠ answer']
df_test_pos

,id,emotion,utterance,cause_utterance,cause_span
2,dailydialog_te_354_utt_2_true_cause_utt_2_span_0,happiness,"Hi there , everyone . We hope you're having a ...","Hi there , everyone . We hope you're having a ...",We hope you're having a good night !
5,dailydialog_te_354_utt_3_true_cause_utt_3_span_0,happiness,Wasn't that football game great ! I just knew ...,Wasn't that football game great ! I just knew ...,Wasn't that football game great ! I just knew ...
8,dailydialog_te_354_utt_4_true_cause_utt_2_span_0,happiness,"The night is young , folks . Get some food and...","Hi there , everyone . We hope you're having a ...",We hope you're having a good night !
9,dailydialog_te_354_utt_4_true_cause_utt_4_span_1,happiness,"The night is young , folks . Get some food and...","The night is young , folks . Get some food and...","The night is young , folks . Get some food and..."
10,dailydialog_tr_9686_utt_2_true_cause_utt_1_span_0,surprise,Me ? You're the one who pulled out in front of...,Why don't you watch where you're going ?,Why don't you watch where you're going ?
...,...,...,...,...,...
3696,dailydialog_te_291_utt_1_true_cause_utt_1_span_0,happiness,I have some great news ! I was able to switch ...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
3698,dailydialog_te_291_utt_2_true_cause_utt_1_span_0,happiness,I knew everything would work out for you !,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
3700,dailydialog_te_291_utt_3_true_cause_utt_1_span_0,happiness,I got to meet the night Anger today too . We g...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
3701,dailydialog_te_291_utt_3_true_cause_utt_3_span_1,happiness,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...


In [17]:
df_pos = pd.concat([df_train_pos, df_valid_pos, df_test_pos])

In [26]:
df_train_pos.emotion.value_counts()

happiness    5721
anger         513
surprise      480
sadness       327
disgust       171
fear           59
Name: emotion, dtype: int64

In [51]:
df_pos = df_pos.reset_index(drop=True)

In [14]:
cause_model_name = 'mrm8488/spanbert-finetuned-squadv2'

spanbert_tokenizer = BertTokenizer.from_pretrained(cause_model_name)
spanbert_model = TFBertModel.from_pretrained(cause_model_name, output_hidden_states = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [58]:
# Manually process data with strange escape characters/mismatching spans to headlines and running the 'nones' list 10 code blocks below
nones_l = [602, 603, 2306, 7534, 8942]
df_proper = df_pos.drop(nones_l)
df_proper

,id,emotion,utterance,cause_utterance,cause_span
0,dailydialog_tr_1097_utt_1_true_cause_utt_1_span_0,happiness,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...
1,dailydialog_tr_1097_utt_3_true_cause_utt_3_span_0,happiness,You have no idea ! There's a banquet with priz...,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances"
2,dailydialog_tr_1097_utt_4_true_cause_utt_3_span_0,surprise,Really ? What kind of prizes ?,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances"
3,dailydialog_tr_1097_utt_5_true_cause_utt_5_span_0,happiness,"Well , I heard that last year Vivian from acco...","Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !
4,dailydialog_tr_1097_utt_6_true_cause_utt_5_span_0,surprise,A new car ! You're kidding !,"Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !
...,...,...,...,...,...
9507,dailydialog_te_291_utt_1_true_cause_utt_1_span_0,happiness,I have some great news ! I was able to switch ...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
9508,dailydialog_te_291_utt_2_true_cause_utt_1_span_0,happiness,I knew everything would work out for you !,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
9509,dailydialog_te_291_utt_3_true_cause_utt_1_span_0,happiness,I got to meet the night Anger today too . We g...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
9510,dailydialog_te_291_utt_3_true_cause_utt_3_span_1,happiness,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...


In [60]:
# Manually adjust cause spans (there weren't too many of these to pose a time concern)
df_improper = df_pos.iloc[nones_l]
cause_improper_list = [
  "haven't seen you in ages",
  "haven't seen you in ages",
  "you will succeed",
  'nineCan I take a message for her ?',
  'they have outspoken colors , own design and the sizes are not always according the mainstream business card .'
  ]
df_improper['cause_span'] = cause_improper_list
df_improper

<ipython-input-60-50d6bcaf0755>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_improper['cause_span'] = cause_improper_list


,id,emotion,utterance,cause_utterance,cause_span
602,dailydialog_tr_784_utt_1_true_cause_utt_1_span_0,surprise,Linda ? Is that you ? I haven't seen you in ag...,Linda ? Is that you ? I haven't seen you in ag...,haven't seen you in ages
603,dailydialog_tr_784_utt_2_true_cause_utt_1_span_0,surprise,Hi George ! It's good to see you !,Linda ? Is that you ? I haven't seen you in ag...,haven't seen you in ages
2306,dailydialog_tr_5481_utt_11_true_cause_utt_11_s...,happiness,You can give it a try ! May be you will succeed .,You can give it a try ! May be you will succeed .,you will succeed
7534,dailydialog_va_201_utt_4_true_cause_utt_3_span_0,happiness,Thank you . But I will call her later,I ' m sorry She isn ' t in at the momment . Sh...,nineCan I take a message for her ?
8942,dailydialog_te_712_utt_4_true_cause_utt_4_span_4,happiness,I agree . Business cards are great tools for p...,I agree . Business cards are great tools for p...,"they have outspoken colors , own design and th..."


In [61]:
df_clean = pd.concat([df_proper, df_improper])
df_clean = df_clean.sort_index()
df_clean

,id,emotion,utterance,cause_utterance,cause_span
0,dailydialog_tr_1097_utt_1_true_cause_utt_1_span_0,happiness,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...
1,dailydialog_tr_1097_utt_3_true_cause_utt_3_span_0,happiness,You have no idea ! There's a banquet with priz...,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances"
2,dailydialog_tr_1097_utt_4_true_cause_utt_3_span_0,surprise,Really ? What kind of prizes ?,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances"
3,dailydialog_tr_1097_utt_5_true_cause_utt_5_span_0,happiness,"Well , I heard that last year Vivian from acco...","Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !
4,dailydialog_tr_1097_utt_6_true_cause_utt_5_span_0,surprise,A new car ! You're kidding !,"Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !
...,...,...,...,...,...
9507,dailydialog_te_291_utt_1_true_cause_utt_1_span_0,happiness,I have some great news ! I was able to switch ...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
9508,dailydialog_te_291_utt_2_true_cause_utt_1_span_0,happiness,I knew everything would work out for you !,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
9509,dailydialog_te_291_utt_3_true_cause_utt_1_span_0,happiness,I got to meet the night Anger today too . We g...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !
9510,dailydialog_te_291_utt_3_true_cause_utt_3_span_1,happiness,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...


In [62]:
# Get tokenized headlines.
c_utt_tokens = spanbert_tokenizer(list(df_clean.cause_utterance))['input_ids']

In [63]:
# Get tokenized cause spans. Remove start and end token.
cause_tokens = spanbert_tokenizer(list(df_clean.cause_span))['input_ids']
cause_tokens = [c[1:-1] for c in cause_tokens]

In [64]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in [i for i,e in enumerate(l) if e==sl[0]]:
        if l[ind:ind+sll]==sl:
            return (ind,ind+sll-1)

In [65]:
start_end_indices = []
for i in range(len(c_utt_tokens)):
  start_end_indices.append(find_sub_list(cause_tokens[i], c_utt_tokens[i]))

In [66]:
nones = [i for (i, e) in enumerate(start_end_indices) if e is None]
nones

[]

In [67]:
df_clean['input_text'] = df_clean.utterance + ' ' + df_clean.cause_utterance

In [83]:
reccon_t_utt_lengths = [spanbert_tokenizer(list(df_clean.utterance)[i],
                                           padding=True,
                                           return_tensors='tf').input_ids.numpy().shape[1] - 1 for i in range(len(df_clean))]

In [85]:
start_indices = []
end_indices = []

for i in range(len(df_clean)):
  start_indices.append(reccon_t_utt_lengths[i] + start_end_indices[i][0] - 1)
  end_indices.append(reccon_t_utt_lengths[i] + start_end_indices[i][1] - 1)

In [88]:
df_clean['start_index'] = start_indices
df_clean['end_index'] = end_indices

In [89]:
df_clean

,id,emotion,utterance,cause_utterance,cause_span,input_text,start_index,end_index
0,dailydialog_tr_1097_utt_1_true_cause_utt_1_span_0,happiness,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,30,51
1,dailydialog_tr_1097_utt_3_true_cause_utt_3_span_0,happiness,You have no idea ! There's a banquet with priz...,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances",You have no idea ! There's a banquet with priz...,27,35
2,dailydialog_tr_1097_utt_4_true_cause_utt_3_span_0,surprise,Really ? What kind of prizes ?,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances",Really ? What kind of prizes ? You have no ide...,13,21
3,dailydialog_tr_1097_utt_5_true_cause_utt_5_span_0,happiness,"Well , I heard that last year Vivian from acco...","Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !,"Well , I heard that last year Vivian from acco...",21,30
4,dailydialog_tr_1097_utt_6_true_cause_utt_5_span_0,surprise,A new car ! You're kidding !,"Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !,"A new car ! You're kidding ! Well , I heard th...",15,24
...,...,...,...,...,...,...,...,...
9507,dailydialog_te_291_utt_1_true_cause_utt_1_span_0,happiness,I have some great news ! I was able to switch ...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !,I have some great news ! I was able to switch ...,23,32
9508,dailydialog_te_291_utt_2_true_cause_utt_1_span_0,happiness,I knew everything would work out for you !,I have some great news ! I was able to switch ...,I was able to switch to the night shift !,I knew everything would work out for you ! I h...,16,25
9509,dailydialog_te_291_utt_3_true_cause_utt_1_span_0,happiness,I got to meet the night Anger today too . We g...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !,I got to meet the night Anger today too . We g...,25,34
9510,dailydialog_te_291_utt_3_true_cause_utt_3_span_1,happiness,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,19,36


In [90]:
df_clean['emotion_label'] = df_clean.emotion.map({'anger':0,
 'disgust':1,
 'fear':2,
 'happiness':3,
 'sadness':4,
 'surprise':5})

In [91]:
df_clean

,id,emotion,utterance,cause_utterance,cause_span,input_text,start_index,end_index,emotion_label
0,dailydialog_tr_1097_utt_1_true_cause_utt_1_span_0,happiness,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,Our company's wei-ya is tomorrow night ! It's ...,30,51,3
1,dailydialog_tr_1097_utt_3_true_cause_utt_3_span_0,happiness,You have no idea ! There's a banquet with priz...,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances",You have no idea ! There's a banquet with priz...,27,35,3
2,dailydialog_tr_1097_utt_4_true_cause_utt_3_span_0,surprise,Really ? What kind of prizes ?,You have no idea ! There's a banquet with priz...,"There's a banquet with prizes , performances",Really ? What kind of prizes ? You have no ide...,13,21,5
3,dailydialog_tr_1097_utt_5_true_cause_utt_5_span_0,happiness,"Well , I heard that last year Vivian from acco...","Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !,"Well , I heard that last year Vivian from acco...",21,30,3
4,dailydialog_tr_1097_utt_6_true_cause_utt_5_span_0,surprise,A new car ! You're kidding !,"Well , I heard that last year Vivian from acco...",last year Vivian from accounting won a new car !,"A new car ! You're kidding ! Well , I heard th...",15,24,5
...,...,...,...,...,...,...,...,...,...
9507,dailydialog_te_291_utt_1_true_cause_utt_1_span_0,happiness,I have some great news ! I was able to switch ...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !,I have some great news ! I was able to switch ...,23,32,3
9508,dailydialog_te_291_utt_2_true_cause_utt_1_span_0,happiness,I knew everything would work out for you !,I have some great news ! I was able to switch ...,I was able to switch to the night shift !,I knew everything would work out for you ! I h...,16,25,3
9509,dailydialog_te_291_utt_3_true_cause_utt_1_span_0,happiness,I got to meet the night Anger today too . We g...,I have some great news ! I was able to switch ...,I was able to switch to the night shift !,I got to meet the night Anger today too . We g...,25,34,3
9510,dailydialog_te_291_utt_3_true_cause_utt_3_span_1,happiness,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,I got to meet the night Anger today too . We g...,19,36,3


In [92]:
# Train, dev, and test split
df_train_preprocessed = df_clean[:7271]
df_valid_preprocessed = df_clean[7271:7618]
df_test_preprocessed = df_clean[7618:]

In [93]:
print(len(df_valid_preprocessed), len(df_test_preprocessed))

347 1894


In [94]:
from google.colab import files
df_train_preprocessed.to_csv('reccon_f1_train_preprocessed.csv')
files.download('reccon_f1_train_preprocessed.csv')
df_valid_preprocessed.to_csv('reccon_f1_valid_preprocessed.csv')
files.download('reccon_f1_valid_preprocessed.csv')
df_test_preprocessed.to_csv('reccon_f1_test_preprocessed.csv')
files.download('reccon_f1_test_preprocessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>